In [ ]:
import time 
import sys
import serial

sys.path.insert(0, 'C:\\Users\\MetsLab\\Documents\\Nina_Python\\metslab') # set path directory 

import reactor_controls
from reactor_controls import instrument_controls as ic
from reactor_controls import weight_control as wc
from reactor_controls import concentration_calc as cc
from reactor_controls import flow_measure as fm

In [ ]:
# get conc replacement amts
# pump #1 will withdraw that volume amount from the reactor
# pump #1 will infuse that into a 'waste' bin
# simultaniuosly as the infuse, pump #2 will withdraw from the solution
# then, pump #2 will infuse into the reactor
# re-calc, and so forth

serW = serial.Serial("COM2") #for Ohaus balance
serW.baudrate = 9600; serW.timeout = 0.1; serW.close()

serP = serial.Serial('COM13') #for SyringeOne pump
serP.baudrate = 19200; serP.timeout=0.1; serP.close()

w_ini = 0
while ii < 3:
    w_ini += ic.getweight(serW)
    time.sleep(10)
    ii += 1

w_ini = w_ini/3

t_ini = time.time()
t_now = time.time()
while (t_now-t_ini) < 300: # Lets this run for 300 seconds 
    pack = cc.x_25_200(serW)
    if type(pack)!= int:
        Vm, Vr, Vn, V1n, V2n = pack
        # Withdraw from reactor, infuse into waste:
        cmd1 = ['vol ml',
                'phn 1','fun rat','rat 0.02 MM','vol %s'%V1n, 'dir wdr',
                'phn 2','fun pas 5',
                'phn 3','fun rat','rat 0.04 MM','vol %s'%V1n, 'dir inf',
                'phn 4','fun stp']
        # Withdraw from solution, infuse into reactor: 
        cmd2 = ['vol ml',
                'phn 1','fun rat','rat 0.02 MM','vol %s'%V1n, 'dir wdr',
                'phn 2','fun pas 5',
                'phn 3','fun rat','rat 0.04 MM','vol %s'%V1n, 'dir inf',
                'phn 4','fun stp']
        for ii in len(cmd1): # Works as long as both have the same # of commands. 
            pumpread(serP,0,cmd1[ii])
            pumpread(serP,1,cmd1[ii])
        pumpread(serP, 0, 'run')
        pumpread(serP, 1, 'run')
    time.sleep(30)
    t_now = time.time()

    
# finally, run the flow_measure code to print amount of Ch4 coming through 